In [ ]:
!rm -rf ~/.cache/huggingface/datasets
!rm -rf ~/.cache/huggingface/hub
!pip install --upgrade --quiet datasets transformers accelerate


In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load the EmpatheticDialogues dataset
dataset = load_dataset("empathetic_dialogues")

# Train/test split
train_ds = dataset["train"]
eval_ds = dataset["validation"] if "validation" in dataset else dataset["test"]

# Load DialoGPT
model_name = "microsoft/DialoGPT-medium"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Fix missing pad token
tokenizer.pad_token = tokenizer.eos_token


In [ ]:
def preprocess_function(examples):
    texts = [
        context + tokenizer.eos_token + response + tokenizer.eos_token
        for context, response in zip(examples["context"], examples["utterance"])
    ]
    tokens = tokenizer(
        texts,
        padding="max_length",
        truncation=True,
        max_length=128,
        return_tensors="pt",
    )
    tokens["labels"] = tokens["input_ids"].clone()
    return tokens


In [ ]:
train_ds = train_ds.map(preprocess_function, batched=True, remove_columns=train_ds.column_names)
eval_ds = eval_ds.map(preprocess_function, batched=True, remove_columns=eval_ds.column_names)

train_ds.set_format(type="torch")
eval_ds.set_format(type="torch")


In [ ]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="steps",
    eval_steps=500,
    save_steps=500,
    save_total_limit=2,
    per_device_train_batch_size=1,      # Very small batch size
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=8,      # Accumulate to simulate batch size 8
    num_train_epochs=3,
    fp16=True,                          # Mixed precision if using GPU
    logging_dir='./logs',
    logging_steps=100,
    report_to=[]                        # Disable W&B to reduce overhead
)


# Data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    tokenizer=tokenizer,
    data_collator=data_collator,
)


In [ ]:
trainer.train()


In [ ]:
model.save_pretrained("./dialoGPT_mental_health")
tokenizer.save_pretrained("./dialoGPT_mental_health")

# Optional: zip and download
!zip -r dialoGPT_mental_health.zip dialoGPT_mental_health
from google.colab import files
files.download("dialoGPT_mental_health.zip")
